In [ ]:
!pip install minio

In [ ]:
import os
from minio import Minio
import requests

In [ ]:
client = Minio("minio-service.kubeflow.svc.cluster.local:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False,           
)

In [ ]:
type(client)

In [ ]:
bucket_name = "newtiledb"

In [ ]:
def upload_files(bucket_name, file_location, client):
    found = False  # Initialize 'found' before the try block
    print("Current working directory:", os.getcwd())
    print("Listing directories in the current working directory:", os.listdir("."))
    print(f"Checking existence of {file_location}: ", os.path.exists(file_location))

    try:
        found = client.bucket_exists(bucket_name)
    except Exception as e:
        print("error trying to search for MinIO Bucket:", e)
        return  # Return early since we cannot proceed without knowing if the bucket exists

    if not found:
        try:
            client.make_bucket(bucket_name)
            print("Created bucket", bucket_name)
        except Exception as e:
            print("Failed to create bucket:", e)
            return  # Return early since we cannot proceed if the bucket cannot be created
    else:
        print("Bucket", bucket_name, "exists, we won't attempt to create one")
        
    # Ensure the directory exists
    if not os.path.isdir(file_location):
        print(f"The directory {file_location} does not exist.")
        return

    # Iterate through all files in the directory
    for file_name in os.listdir(file_location):
        # Construct the full file path
        source_file = os.path.join(file_location, file_name)
        # Check if it's a file and not a directory
        if os.path.isfile(source_file):
            try:
                # Upload the file
                client.fput_object(bucket_name, file_name, source_file)
                print(f"Successfully uploaded {file_name} to bucket {bucket_name}.")
            except Exception as e:
                print(f"Failed to upload {file_name}: {e}")


In [219]:
data = {
  "instances": [{
    "input": "When was Kubeflow Open-Sourced?",
    "num_docs": 6  # number of documents to retrieve
  }]
}

URL = "http://vectorstore-predictor.christensenc3526.svc.cluster.local/v1/models/vectorstore:predict"  # Adjust path as necessary

response = requests.post(URL, json=data, verify=False)  # 'verify=False' for self-signed certs
print(response)
#print(response.json())
print(response.text)

<Response [200]>
{"predictions":["about Kubeflow on Nutanix, see GitHub.","Chapter 4.  The Origin of Kubeflow \nChapter Introduction\nChapter Overview","an open source project that welcomes community contributions, suggestions, fixes, and constructive feedback. You can contribute on GitHub and get started by following the instructions in the Canonical Kubeflow documentation.","Kubeflow’s Technical Advantage","Kubeflow on Red Hat OpenShift","consider themselves distributions. These Kubeflow-centric solutions range from purely open source offerings to open core offerings with premium integrations. The community has not officially endorsed any of these distributions yet. Still, based on current in-house skillsets or other considerations, organizations can speak with the distribution maintainers and make educated decisions about how they want to deploy and maintain their Kubeflow environments."]}


In [213]:
!curl -X GET http://llm-predictor.christensenc3526.svc.cluster.local/v1/models

{"object":"list","data":[{"id":"meta/llama-2-7b-chat","object":"model","created":1738273856,"owned_by":"system","root":"meta/llama-2-7b-chat","parent":null,"permission":[{"id":"modelperm-4e4ef512cf0c432b81a0057a7b6488a3","object":"model_permission","created":1738273856,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [214]:
!curl -X GET http://llm.christensenc3526.svc.cluster.local/v1/models

{"models":["llm"]}

# run this command from your terminal
```
curl -X POST http://llm-predictor.christensenc3526.svc.cluster.local/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model": "meta/llama-2-7b-chat",
    "messages": [{"role": "user", "content": "What is KServe?"}],
    "temperature": 0.5,
    "top_p": 1,
    "max_tokens": 256,
    "stream": false
  }'
``` 

In [224]:
data = {
  "instances": [{
    "input": "When was Kubeflow Open-Sourced?",
    "num_docs": 6  # number of documents to retrieve
  }]
}

URL = "http://llm.christensenc3526.svc.cluster.local/v1/models/llm:predict"  # Adjust path as necessary

response = requests.post(URL, json=data, verify=False)  # 'verify=False' for self-signed certs
print(response)
#print(response.json())
print(response.text)

<Response [500]>
{"error":"HTTPStatusError : {'detail': 'Not Found'}, '404 Not Found' for url 'http://llm-predictor.christensenc3526/v1/models/llm:predict'"}


In [ ]:
this works 